In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pylab  as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV,  ParameterGrid
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectKBest
from sklearn.dummy import DummyClassifier
from sklearn.utils.multiclass import unique_labels


In [2]:
features = pd.read_csv('../data/features_plus_descriptions.csv')
features.set_index('Feature Type and Number', inplace=True)
features.drop(['S5', 'D21'], axis=0, inplace=True)

In [3]:
df = pd.read_csv('../data/multiclass_raw_data.csv')
X = df.loc[:,features.index]
y = df['Best Heuristic']

In [4]:
types = dict.fromkeys(features.index, 'stdsc')
mm_feats = ['S1','S3', 'S4', 'S6', 'S8','S11', 'S12', 'D3', 'D39']

for feat in mm_feats:
    types[feat] = 'minmax'

minmax_feats = []
std_feats = []

for feat in types.keys():
    if types[feat] == 'stdsc':
        std_feats.append(feat)
    elif types[feat] == 'minmax':
        minmax_feats.append(feat)

print('MinMax Scaler Features: ', minmax_feats)
print('Standard Scaler Features: ', std_feats)

preprocessor = ColumnTransformer(
    transformers=[
        ('mm_scaler', MinMaxScaler(), minmax_feats),
        ('std_scaler', StandardScaler(), std_feats)])

MinMax Scaler Features:  ['S1', 'S3', 'S4', 'S6', 'S8', 'S11', 'S12', 'D3', 'D39']
Standard Scaler Features:  ['S2', 'S7', 'S9', 'S10', 'S13', 'S14', 'D1', 'D2', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38']


In [7]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['figure.figsize'] = (8,8)
    plt.rcParams['font.size'] = 8
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
  #  classes = np.array(classes)
  #  classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [12]:
def ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid, verbose=5):
    """
    This function splits the data into other and test sets (80-20 split) and
    then applies KFold with 4 folds to other set. 
    """
    prec_scores = []
    rec_scores = []
    f1_scores = []
    best_models = []
    ys = []
    for i in range(10):
        random_state = 431*i
        X_other, X_test, y_other, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
        kf = KFold(n_splits=4, shuffle=True, random_state=random_state)
        clf = Pipeline([
        ('preprocessor', preprocessor), 
        ('clf', ML_algo)
        ])
        grid = GridSearchCV(clf, param_grid, scoring='f1_macro',cv=kf,  return_train_score=True, verbose=verbose, n_jobs=-2)  
        grid.fit(X_other, y_other)
        y_pred = grid.predict(X_test)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")
        best_models.append(grid.best_params_)
        print("best params", grid.best_params_, "precision score", prec, "recall score", rec, "f1 score", f1)
        prec_scores.append(prec)
        rec_scores.append(rec)
        f1_scores.append(f1)
        ys.append((y_test, y_pred, [0,1,2,3,4,5]))
    return [prec_scores, rec_scores, f1_scores], best_models, ys


In [14]:
ML_algo = DummyClassifier(strategy="uniform")
param_grid = { 'clf__strategy':  ["stratified", "most_frequent", "prior", "uniform", "constant"] }

scores_dummy, models_dummy, ys_dummy = ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid)

names = ['precision', 'recall', 'f1 score']

for i, s in enumerate(scores_dummy):
    print('dummy {}:'.format(names[i]), np.round(np.mean(s), decimals=4), '+/-', np.round(np.std(s), decimals=4))

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done  12 out of  20 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-2)]: Done  17 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  20 out of  20 | elapsed:    0.2s finished


best params {'clf__strategy': 'stratified'} precision score 0.16085250607295476 recall score 0.1592900141426715 f1 score 0.15969082402156906
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done  12 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-2)]: Done  17 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  20 out of  20 | elapsed:    0.1s finished


best params {'clf__strategy': 'stratified'} precision score 0.17741591916081279 recall score 0.17696492699534658 f1 score 0.17673945704676464
Fitting 4 folds for each of 5 candidates, totalling 20 fits


...

best params {'clf__strategy': 'stratified'} precision score 0.1758979663085111 recall score 0.17589659314300934 f1 score 0.17537161576641316
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done  12 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-2)]: Done  17 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  20 out of  20 | elapsed:    0.2s finished


best params {'clf__strategy': 'stratified'} precision score 0.17048505804852213 recall score 0.16969996949310326 f1 score 0.1693875489843751
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done  12 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-2)]: Done  17 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  20 out of  20 | elapsed:    0.1s finished


best params {'clf__strategy': 'stratified'} precision score 0.16058003945184876 recall score 0.1597304678691994 f1 score 0.16002044564388265
dummy precision: 0.1657 +/- 0.0089
dummy recall: 0.1657 +/- 0.0088
dummy f1 score: 0.1653 +/- 0.0087


In [20]:
param_grid = { 'clf__max_depth': [15, 20, 25, 30, 35, 40], 
               'clf__max_features':[15, 20, 25, 30, 35, 40, 45, 51] }

ML_algo = RandomForestClassifier()
pg = ParameterGrid(param_grid)
scores_RF, models_RF, ys_RF = ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid)

Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done 192 out of 192 | elapsed:  2.4min finished


best params {'clf__max_depth': 20, 'clf__max_features': 25} precision score 0.5284413291334079 recall score 0.5097200982486344 f1 score 0.516700439325986
Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done 192 out of 192 | elapsed:  2.4min finished


best params {'clf__max_depth': 25, 'clf__max_features': 30} precision score 0.5470399218744088 recall score 0.5186353845465365 f1 score 0.5286331742512159
Fitting 4 folds for each of 48 candidates, totalling 192 fits


...

best params {'clf__max_depth': 30, 'clf__max_features': 45} precision score 0.5295159419766651 recall score 0.508415699551794 f1 score 0.5169835598288149
Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done 192 out of 192 | elapsed:  2.4min finished


best params {'clf__max_depth': 20, 'clf__max_features': 20} precision score 0.5250625593898829 recall score 0.4948143336774307 f1 score 0.5062824418156
Fitting 4 folds for each of 48 candidates, totalling 192 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done 192 out of 192 | elapsed:  2.4min finished


best params {'clf__max_depth': 20, 'clf__max_features': 25} precision score 0.5263543973415287 recall score 0.48984054928244336 f1 score 0.5040048195920903


In [22]:
names = ['precision', 'recall', 'f1 score']

for i, s in enumerate(scores_RF):
    print('Random Forest {}:'.format(names[i]), np.round(np.mean(s), decimals=4), '+/-', np.round(np.std(s), decimals=4))
    for j in range(len(s)):
        print('     ', s[j])
for m in models_RF: 
    print(m)


Random Forest precision: 0.5346 +/- 0.009
      0.5284413291334079
      0.5470399218744088
      0.5421965693996346
      0.5287297987502864
      0.5340548931541579
      0.5323666295582387
      0.5527291521765695
      0.5295159419766651
      0.5250625593898829
      0.5263543973415287
Random Forest recall: 0.5085 +/- 0.0093
      0.5097200982486344
      0.5186353845465365
      0.5152330835905438
      0.51139944343931
      0.5046988108530209
      0.5110985862187565
      0.5211241774028249
      0.508415699551794
      0.4948143336774307
      0.48984054928244336
Random Forest f1 score: 0.5187 +/- 0.0088
      0.516700439325986
      0.5286331742512159
      0.525201154817523
      0.518213518373923
      0.5165674937703708
      0.5197478859179984
      0.534249291824065
      0.5169835598288149
      0.5062824418156
      0.5040048195920903
{'clf__max_depth': 20, 'clf__max_features': 25}
{'clf__max_depth': 25, 'clf__max_features': 30}
{'clf__max_depth': 30, 'clf__max_featur

In [26]:
print(np.argmax(scores_RF[2]), np.max(scores_RF[2]))
print(models_RF[np.argmax(scores_RF[2])])

6 0.534249291824065
{'clf__max_depth': 35, 'clf__max_features': 45}


In [28]:
param_grid ={ 'clf__n_neighbors': [5,10,15,20,30,40,50,60],
              'clf__weights': ['distance', 'uniform'] }
ML_algo = KNeighborsClassifier()
scores_KNN, models_KNN, ys_KNN = ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid)


Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    8.8s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    8.8s finished


best params {'clf__n_neighbors': 20, 'clf__weights': 'distance'} precision score 0.49446774037709823 recall score 0.46972054688762527 f1 score 0.4780951095144126
Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    7.6s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    7.6s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


best params {'clf__n_neighbors': 15, 'clf__weights': 'distance'} precision score 0.5186984539287631 recall score 0.47958883730297 f1 score 0.4921345925425109
Fitting 4 folds for each of 16 candidates, totalling 64 fits


...

[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    7.9s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    7.9s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


best params {'clf__n_neighbors': 10, 'clf__weights': 'distance'} precision score 0.5119390922281565 recall score 0.4854172482118911 f1 score 0.49503304874633575
Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.6s


best params {'clf__n_neighbors': 15, 'clf__weights': 'distance'} precision score 0.4929639166622854 recall score 0.45397255174239054 f1 score 0.4674086409837887


[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done  64 out of  64 | elapsed:    8.2s finished
